In [1]:
from miditoolkit.midi.parser import MidiFile

from deepnote import MusicRepr, Constants, Metric, Note, utils

## constants

In [2]:
const = Constants(
    unit=4,
    min_tempo=30,
    max_tempo=300,
    num_tempo_bins=30, 
    num_velocity_bins=30
)
const

Constants(unit=4, tick_resol=120, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)

## midi representation

In [3]:
# seq = MusicRepr.from_file(
#     '../test-1.mid', 
#     unit=4,
#     min_tempo=30,
#     max_tempo=300,
#     num_tempo_bins=30, 
#     num_velocity_bins=30
# )
seq = MusicRepr.from_file('assets/test.mid', const=const)
seq

MusicRepr(
 num_events=4307,
 const=Constants(unit=4, tick_resol=120, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)
)

In [4]:
seq.const

Constants(unit=4, tick_resol=120, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)

In [5]:
seq[:10]

[Bar(position=0, tempo=123),
 Beat(position=9, tempo=104),
 Bar(position=0),
 Beat(position=4),
 Note(inst_family=piano, pitch=64, duration=1, velocity=105),
 Beat(position=5),
 Note(inst_family=piano, pitch=65, duration=1, velocity=87),
 Beat(position=7),
 Note(inst_family=piano, pitch=67, duration=3, velocity=87),
 Beat(position=11)]

### splitting bars

In [6]:
seq.get_bar_count()

76

In [10]:
bars = seq.get_bars()
len(bars)

76

## concatenating bars

In [9]:
seq2 = MusicRepr.concatenate(bars[:2])
seq2.get_bar_count()

2

## Slicing

### bar index

In [11]:
seq2 = seq.slice_by_bar(1,2)
print(seq2.events == bars[1].events, seq2 == bars[1])

True True


### beat index

In [ ]:
seq2 = seq.slice_by_beat(12, 16)
print(seq2.events == bars[3].events, seq2 == bars[3])

### event index

In [15]:
seq2 = seq.slice_by_index(0,10)
seq2[:]

[Bar(position=0, tempo=123),
 Beat(position=9, tempo=104),
 Bar(position=0),
 Beat(position=4),
 Note(inst_family=piano, pitch=64, duration=1, velocity=105),
 Beat(position=5),
 Note(inst_family=piano, pitch=65, duration=1, velocity=87),
 Beat(position=7),
 Note(inst_family=piano, pitch=67, duration=3, velocity=87),
 Beat(position=11)]

## separating different instrument tracks

In [16]:
tracks = seq.separate_tracks()
list(tracks.keys()), seq.get_instruments()

(['ensemble', 'piano', 'drums', 'bass', 'guitar'],
 ['ensemble', 'piano', 'drums', 'bass', 'guitar'])

In [17]:
for inst in tracks:
    print(tracks[inst].get_bar_count())

76
76
76
76
76


In [18]:
tracks['bass'][:10]

[Bar(position=0, tempo=123),
 Beat(position=9, tempo=104),
 Bar(position=0),
 Bar(position=0, chord=G_M7),
 Note(inst_family=bass, pitch=43, duration=8, velocity=74),
 Beat(position=8, chord=B_m),
 Note(inst_family=bass, pitch=50, duration=8, velocity=74),
 Bar(position=0, chord=G_M7),
 Note(inst_family=bass, pitch=43, duration=8, velocity=79),
 Beat(position=8, chord=G_7)]

## exporting

### midi

In [19]:
midi = seq.to_midi('assets/rec_test.mid')
midi

ticks per beat: 120
max tick: 36390
tempo changes: 2
time sig: 1
key sig: 0
markers: 70
lyrics: False
instruments: 5

### remi

In [20]:
remi = seq.to_remi(ret='token')
remi[:10]

['Bar',
 'BeatTempo_123',
 'BeatPosition_9',
 'BeatTempo_104',
 'Bar',
 'BeatPosition_4',
 'NoteInstFamily_piano',
 'NotePitch_64',
 'NoteDuration_1',
 'NoteVelocity_105']

In [21]:
seq.to_remi(ret='event')[:10]

[Event(type=Bar),
 Event(type=BeatTempo, value=123),
 Event(type=BeatPosition, value=9),
 Event(type=BeatTempo, value=104),
 Event(type=Bar),
 Event(type=BeatPosition, value=4),
 Event(type=NoteInstFamily, value=piano),
 Event(type=NotePitch, value=64),
 Event(type=NoteDuration, value=1),
 Event(type=NoteVelocity, value=105)]

In [22]:
seq.to_remi(ret='index')[:10]

[0, 28, 11, 26, 0, 6, 180, 261, 325, 365]

### compound word

In [23]:
cp = seq.to_cp()
cp.shape, len(seq)

((4307, 8), 4307)

In [24]:
bars[0].to_cp()

array([[ 0,  0, 11,  0,  0,  0,  0,  0],
       [ 0,  9,  9,  0,  0,  0,  0,  0]])

## Importing

### from cp

In [25]:
seq_t = MusicRepr.from_cp(cp, const=seq.const)
seq_t.events == seq.events, seq == seq_t

(True, True)

### from remi

In [27]:
remi

['Bar',
 'BeatTempo_123',
 'BeatPosition_9',
 'BeatTempo_104',
 'Bar',
 'BeatPosition_4',
 'NoteInstFamily_piano',
 'NotePitch_64',
 'NoteDuration_1',
 'NoteVelocity_105',
 'BeatPosition_5',
 'NoteInstFamily_piano',
 'NotePitch_65',
 'NoteDuration_1',
 'NoteVelocity_87',
 'BeatPosition_7',
 'NoteInstFamily_piano',
 'NotePitch_67',
 'NoteDuration_3',
 'NoteVelocity_87',
 'BeatPosition_11',
 'NoteInstFamily_piano',
 'NotePitch_64',
 'NoteDuration_3',
 'NoteVelocity_74',
 'BeatPosition_13',
 'NoteInstFamily_piano',
 'NotePitch_65',
 'NoteDuration_1',
 'NoteVelocity_79',
 'BeatPosition_14',
 'NoteInstFamily_piano',
 'NotePitch_64',
 'NoteDuration_1',
 'NoteVelocity_96',
 'Bar',
 'BeatChord_G_M7',
 'NoteInstFamily_bass',
 'NotePitch_43',
 'NoteDuration_8',
 'NoteVelocity_74',
 'NoteInstFamily_piano',
 'NotePitch_43',
 'NoteDuration_1',
 'NoteVelocity_66',
 'NoteInstFamily_piano',
 'NotePitch_64',
 'NoteDuration_1',
 'NoteVelocity_96',
 'NoteInstFamily_drums',
 'NotePitch_35',
 'NoteDuration

In [26]:
seq_t = MusicRepr.from_string(' '.join(remi), const=seq.const)
seq_t.events == seq.events, seq == seq_t

AssertionError: 

## merging tracks

In [25]:
seq_t = MusicRepr.merge_tracks(tracks)
seq == seq_t

True